In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import cv2
import numpy as np
from PIL import Image
import os
import pandas as pd
from datetime import datetime

# Global user data storage (to map user ID to names dynamically)
user_data = {}  
user_id_counter = 1  # Auto-increment ID for users

# Function to center the window
def center_window(win, width=800, height=300):
    screen_width = win.winfo_screenwidth()
    screen_height = win.winfo_screenheight()
    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)
    win.geometry(f"{width}x{height}+{x}+{y}")

# Create the main window
window = tk.Tk()
window.title("Face Recognition & Attendance System")
window.configure(bg="#f0f0f0")
center_window(window)

# Create a stylish frame
frame = tk.Frame(window, bg="white", padx=20, pady=20, relief="raised", borderwidth=2)
frame.pack(pady=20)

# Labels and Entry Fields
style = ("Helvetica", 16)

l1 = tk.Label(frame, text="Name:", font=style, bg="white")
l1.grid(column=0, row=0, sticky="w", pady=5, padx=10)
t1 = ttk.Entry(frame, width=40)
t1.grid(column=1, row=0, pady=5, padx=10)

# Buttons
button_style = {"font": ("Helvetica", 14, "bold"), "padx": 10, "pady": 5, "width": 18}

# Function to generate a dataset
def generate_dataset():
    global user_id_counter

    name = t1.get()
    if not name:
        messagebox.showinfo("Result", "Please provide a name.")
        return

    # Assign an ID and store user data
    user_data[user_id_counter] = name  

    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None
        
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]  

    cap = cv2.VideoCapture(0)  
    img_id = 0  

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cropped_face = face_cropped(frame)

        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"data/user.{user_id_counter}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or img_id == 200:  
            break

    cap.release()
    cv2.destroyAllWindows()

    messagebox.showinfo("Result", f"Dataset Generated for {name} (ID: {user_id_counter})!")
    user_id_counter += 1  

b1 = tk.Button(frame, text="Generate Dataset", **button_style, bg="#2196F3", fg="white", command=generate_dataset)
b1.grid(column=0, row=2, pady=15, padx=5)

# Function to train classifier
def train_classifier():
    data_dir = "data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces, ids = [], []

    for image in path:
        img = Image.open(image).convert("L")
        imageNp = np.array(img, "uint8")
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)  

    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    
    messagebox.showinfo("Result", "Training Completed!")

b2 = tk.Button(frame, text="Train Model", **button_style, bg="#4CAF50", fg="white", command=train_classifier)
b2.grid(column=1, row=2, pady=15, padx=5)

# Function for face detection and attendance marking
def detect_face():
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    attendance_file = "attendance.xlsx"
    if not os.path.exists(attendance_file):
        df = pd.DataFrame(columns=["ID", "Name", "Time"])
        df.to_excel(attendance_file, index=False)

    def mark_attendance(user_id, name):
        df = pd.read_excel(attendance_file)
        today = datetime.now().date()
        if not ((df["ID"] == user_id) & (pd.to_datetime(df["Time"]).dt.date == today)).any():
            new_entry = pd.DataFrame([[user_id, name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")]], 
                                     columns=["ID", "Name", "Time"])
            df = pd.concat([df, new_entry], ignore_index=True)
            df.to_excel(attendance_file, index=False)

    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            id, pred = clf.predict(gray_img[y:y+h, x:x+w])
            confidence = 100 - pred  

            if confidence > 45:  
                name = user_data.get(id, "Unknown")  
                cv2.putText(img, f"{name} ({confidence:.2f}%)", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                if name != "Unknown":
                    mark_attendance(id, name)
            else:
                cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

        return img

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        if not ret:
            break

        img = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), clf)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) == 13:  
            break

    video_capture.release()
    cv2.destroyAllWindows()

b3 = tk.Button(frame, text="Detect Faces", **button_style, bg="#FF9800", fg="white", command=detect_face)
b3.grid(column=2, row=2, pady=15, padx=5)

window.mainloop()
